In [105]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import csv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
!pip3 install idx2numpy
import idx2numpy

In [106]:
arr_img = idx2numpy.convert_from_file('/content/train-images.idx3-ubyte')
arr_labels = idx2numpy.convert_from_file('/content/train-labels.idx1-ubyte')
arr_img = np.reshape(arr_img, (60000, 784))

In [107]:
pca15 = PCA(n_components=15)
pca15.fit(arr_img)

PCA(n_components=15)

In [108]:
arr_trans = pca15.transform(arr_img)
print((arr_trans[0]))

[ 123.93258849 -312.67425848  -24.51404645 -555.75794895  -27.29673749
 -232.47615477  233.99511676  159.8109179  -363.52338563  198.44334755
  197.47539601 -253.98464895 -113.72005345  749.24721568  218.3974841 ]


In [112]:
classwise = []
for i in range(len(np.unique(arr_labels))):
  mid = []
  for j in range(len(arr_labels)):
    if (arr_labels[j]==i):
      mid.append(list((arr_trans[j])))
  classwise.append(mid)

In [113]:
#global variables for better readability
num_class = len(classwise)
num_pixels = len(classwise[0][0])

In [114]:
means = []
for i in range(num_class):
  means.append(np.mean(classwise[i], axis = 0))

In [115]:
wscatters = []
for i in range(num_class):
  sum = [[0]*num_pixels for j in range(num_pixels)]
  for j in range(len(classwise[i])):
    sum+= np.matmul(np.transpose(means[i] - classwise[i][j]),means[i] - classwise[i][j])
  wscatters.append(sum)

In [116]:
sw = wscatters[0]
for i in range(1, num_class):
  sw+=wscatters[i]

In [153]:
big_mean = np.mean(arr_trans, axis=0)

In [118]:
sb = [[0]*num_pixels for i in range(num_pixels)]
for i in range(num_class):
  sb+=len(classwise[i])*np.matmul(np.transpose(means[i]-big_mean),means[i]-big_mean)

In [121]:
mat = np.matmul(np.linalg.pinv(sw),sb)

In [143]:
val, vec = np.linalg.eigh(mat)
idx = val.argsort()[-15:][::-1]
val = val[idx]
vec = vec[idx]
w = vec

In [144]:
y = []
for xi in arr_trans:
  xi = np.transpose(xi)
  xi = np.asarray(xi, dtype='float64')
  y.append(np.transpose((np.matmul(w, xi))))

In [145]:
test_labels = idx2numpy.convert_from_file('/content/t10k-labels.idx1-ubyte')

In [146]:
x_test = idx2numpy.convert_from_file('/content/t10k-images.idx3-ubyte')
x_test = np.reshape(x_test, (10000, 784))
y_test = []
x_test = pca15.transform(x_test)
for xi in x_test:
  xi = np.transpose(xi)
  xi = np.asarray(xi, dtype='float64')
  y_test.append(np.transpose(np.matmul(w, xi)))

In [147]:
clf = LDA()
clf.fit(y, arr_labels)

LinearDiscriminantAnalysis()

In [154]:
pred = clf.predict(y_test)

In [155]:
correct = 0
incorrect = 0
for i in range(len(pred)):
  if (pred[i]==test_labels[i]):
    correct+=1
  else:
    incorrect+=1
print("Accuracy = "+ str(correct*100/(correct+incorrect)))

Accuracy = 80.4


In [156]:
for i in range(num_class):
  correct = 0
  incorrect = 0
  for j in range(len(pred)):
    if (test_labels[j] == i and pred[j]==test_labels[j]):
      correct+=1
    elif (test_labels[j] == i):
      incorrect+=1
  print("Accuracy of class "+str(i)+" = "+str(correct*100/(correct+incorrect)))

Accuracy of class 0 = 90.3061224489796
Accuracy of class 1 = 91.54185022026432
Accuracy of class 2 = 73.93410852713178
Accuracy of class 3 = 79.3069306930693
Accuracy of class 4 = 81.16089613034623
Accuracy of class 5 = 64.79820627802691
Accuracy of class 6 = 82.6722338204593
Accuracy of class 7 = 83.852140077821
Accuracy of class 8 = 74.84599589322382
Accuracy of class 9 = 78.69177403369673
